# 1) Import Libraries




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9451db5b6d2fb82befe677c532b366c0ba96e54875ef0dee2c5430ccc12e11de
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import os
import torch
from tqdm import tqdm
import evaluate

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
MAX_LENGTH=300
BATCH_SIZE=16

# 2) Load the tokenizer and the model.

In [7]:
tokenizer=AutoTokenizer.from_pretrained('distilgpt2', add_special_tokens=True,additional_special_tokens=['[response]'], pad_token="[Pad]",padding_side='left')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [19]:
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
model.resize_token_embeddings(len(tokenizer))
model=model.to(device)

In [8]:
bos_token=tokenizer.decode(tokenizer.bos_token_id)
eos_token=tokenizer.decode(tokenizer.eos_token_id)

# 3) Evaluate on Combined Dataset

## 3.1) Create the Dataset and the Dataloader.

In [16]:
class MedicalDataset(Dataset):
    def __init__(self, tokenizer, questions, answers):
        self.tokenizer = tokenizer
        self.tokenized_questions=self.tokenizer(questions, padding=True, truncation=False, return_tensors='pt')
        self.answers=answers

    def __len__(self):
        return len(self.tokenized_questions['input_ids'])

    def __getitem__(self, idx):
        return self.tokenized_questions['input_ids'][idx], self.tokenized_questions['attention_mask'][idx], self.answers[idx]


In [17]:
dataset_pd=pd.read_csv(f'/content/drive/MyDrive/NLP_Project/test_datasets/combined_test.csv')
dataset_pd=dataset_pd.astype('string')


In [18]:
test_dataset = MedicalDataset(tokenizer ,list(dataset_pd['Question']), list(dataset_pd['Answer_cut']))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## 3.2) Model Evaluation

- Get the Bleu and Rouge scores on Test Dataset

In [24]:
model.eval()
bleu=evaluate.load('sacrebleu')
rouge=evaluate.load('rouge')
for input_ids, attention_mask, answers in tqdm(test_loader):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask,  pad_token_id=tokenizer.pad_token_id,max_new_tokens=MAX_LENGTH)
    decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    for pred, ref in zip(answers, decoded_outputs):
        bleu.add(references = ref, predictions=pred)
        rouge.add(references = ref, predictions=pred)
bleu=bleu.compute()
rouge=rouge.compute()

print(f"Bleu: {bleu['score']},  Rouge:{rouge['rouge1']}, Rouge:{rouge['rougeL']}")

100%|██████████| 577/577 [18:07<00:00,  1.89s/it]


Bleu: 0.891,  Rouge:0.12, Rouge:0.086


# 4) Evaluate on MedQuaD Dataset

## 4.1) Create the Dataset and the Dataloader.

In [32]:
dataset_pd=pd.read_csv(f'/content/drive/MyDrive/NLP_Project/test_datasets/MedQuAD_test.csv')
dataset_pd=dataset_pd.astype('string')

In [33]:
test_dataset = MedicalDataset(tokenizer ,list(dataset_pd['Question']), list(dataset_pd['Answer_cut']))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## 4.2) Model Evaluation

In [36]:
model.eval()
bleu=evaluate.load('sacrebleu')
rouge=evaluate.load('rouge')
for input_ids, attention_mask, answers in tqdm(test_loader):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask,  pad_token_id=tokenizer.pad_token_id,max_new_tokens=MAX_LENGTH)
    decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    for pred, ref in zip(answers, decoded_outputs):
        bleu.add(references = ref, predictions=pred)
        rouge.add(references = ref, predictions=pred)
bleu=bleu.compute()
rouge=rouge.compute()

print(f"Bleu: {bleu['score']},  Rouge:{rouge['rouge1']}, Rouge:{rouge['rougeL']}")

100%|██████████| 205/205 [06:03<00:00,  1.77s/it]


Bleu: 0.717,  Rouge:0.076, Rouge:0.067


# 5) Evaluate on iCliniq Dataset

## 5.1) Create the Dataset and the Dataloader.

In [37]:
dataset_pd=pd.read_csv(f'/content/drive/MyDrive/NLP_Project/test_datasets/iCliniq_test.csv')
dataset_pd=dataset_pd.astype('string')

In [38]:
test_dataset = MedicalDataset(tokenizer ,list(dataset_pd['Question']), list(dataset_pd['Answer_cut']))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## 5.2) Model Evaluation

In [43]:
model.eval()
bleu=evaluate.load('sacrebleu')
rouge=evaluate.load('rouge')
for input_ids, attention_mask, answers in tqdm(test_loader):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask,  pad_token_id=tokenizer.pad_token_id,max_new_tokens=MAX_LENGTH)
    decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    for pred, ref in zip(answers, decoded_outputs):
        bleu.add(references = ref, predictions=pred)
        rouge.add(references = ref, predictions=pred)
bleu=bleu.compute()
rouge=rouge.compute()

print(f"Bleu: {bleu['score']},  Rouge:{rouge['rouge1']}, Rouge:{rouge['rougeL']}")

100%|██████████| 372/372 [11:37<00:00,  1.88s/it]


Bleu: 0.213,  Rouge:0.116, Rouge:0.084
